In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')
pd.pandas.set_option('display.max_columns', None)

In [4]:
import sys
def return_size(df):
    """Return size of dataframe in gigabytes"""
    return round(sys.getsizeof(df) / 1e9, 2)

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df

In [23]:
train = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')
bureau = pd.read_csv('bureau.csv')
bureau_bal = pd.read_csv('bureau_balance.csv')
pos_cash_bal = pd.read_csv('POS_CASH_balance.csv')
credit_card_bal = pd.read_csv('credit_card_balance.csv')
previous_app = pd.read_csv('previous_application.csv')
installment_payments = pd.read_csv('installments_payments.csv')

In [24]:
train = convert_types(train)
test = convert_types(test)
bureau = convert_types(bureau)
bureau_bal = convert_types(bureau_bal)
pos_cash_bal = convert_types(pos_cash_bal)
credit_card_bal = convert_types(credit_card_bal)
previous_app = convert_types(previous_app)
installment_payments = convert_types(installment_payments)

In [25]:
def missing(df):
    miss_val = df.isnull().sum()
    miss_val_perc = 100 * df.isnull().sum()/len(df)
    miss_val_table = pd.concat([miss_val, miss_val_perc], axis = 1)
    miss_val_table_rename = miss_val_table.rename(columns = {0: 'Missing Values', 1: '% of Total Values'})
    miss = miss_val_table_rename[miss_val_table_rename['Missing Values'] != 0].sort_values('% of Total Values', ascending = False).round(1)
    print("Your dataframe has "+ str(df.shape[1]) + " columns out of which " + str(miss.shape[0]) + " columns have missing values.")
    return miss

In [26]:
def agg_fn_numeric(data, col, name):
    df = data.copy()
    for column in df:
        if column != col and 'SK_ID' in column:
            df.drop(column, axis = 1, inplace = True)
    
    group_ids = df[col]
    numeric_df = df.select_dtypes('number')
    numeric_df[col] = group_ids        
    
    agg = numeric_df.groupby(col).agg(['count','mean', 'max', 'min','sum']).reset_index()
    
    c = [col]
    for var in agg.columns.levels[0]:
        if var != col:
            for stat in agg.columns.levels[1][:-1]:
                c.append('{}_{}_{}'.format(name.upper(), var.upper(), stat.upper()))
    agg.columns = c
    return agg

In [40]:
def agg_fn_categorical(data, col, name):
    df = data.copy()
    for column in df:
        if column != col and 'SK_ID' in column:
            df = df.drop(column, axis = 1)
        
    group_ids = df[col]
    categorical_df = pd.get_dummies(df.select_dtypes('category'))
    categorical_df[col] = group_ids 

    agg = categorical_df.groupby(col).agg(['sum', 'mean'])
    
    columns = []
    for var in agg.columns.levels[0]:
        for stat in ['COUNT', 'COUNT_NORM']:
            columns.append('{}_{}_{}'.format(name.upper(), var.upper(), stat))

    agg.columns = columns
    return agg



In [41]:
def double_agg_fn(data, ids, names, id_df):
    agg_conti = agg_fn_numeric(data, ids[1], names[1])
    agg_cat = agg_fn_categorical(data, ids[1], names[1])
    agg_merged = agg_cat.merge(agg_conti, on = ids[1], how = 'outer')
    temp = agg_merged.merge(id_df[[ids[1], ids[0]]], how = 'left', on = ids[1])
    final = agg_fn_numeric(temp.drop(ids[1], axis = 1), ids[0], names[0])
    return final

In [55]:
bureau_agg_conti = agg_fn_numeric(data = bureau.drop('SK_ID_BUREAU', axis = 1), col = 'SK_ID_CURR', name = 'BUREAU')
bureau_agg_cat = agg_fn_categorical(data = bureau, col = 'SK_ID_CURR', name = 'BUREAU')



<ipython-input-26-bcb3b9189408>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[col] = group_ids


In [56]:
bureau_final = bureau_agg_cat.merge(bureau_agg_conti, on = 'SK_ID_CURR', how = 'outer')

In [47]:
bureau_bal_agg_conti = agg_fn_numeric(bureau_bal, 'SK_ID_BUREAU', 'BUREAU_BAL')
bureau_bal_agg_cat = agg_fn_categorical(bureau_bal, 'SK_ID_BUREAU', 'BUREAU_BAL')

<ipython-input-26-bcb3b9189408>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[col] = group_ids


In [49]:
bureau_bal_agg_merged = bureau_bal_agg_cat.merge(bureau_bal_agg_conti, on = 'SK_ID_BUREAU', how = 'outer')

In [52]:
bureau_bal_agg = bureau_bal_agg_merged.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], how = 'left', on = 'SK_ID_BUREAU')

In [53]:
bureau_bal_final = agg_fn_numeric(bureau_bal_agg.drop('SK_ID_BUREAU', axis = 1), 'SK_ID_CURR', 'CLIENT')

In [61]:
bureau_bal_final.shape

(134542, 106)

In [64]:
prev_app_agg_conti = agg_fn_numeric(data = previous_app.drop('SK_ID_PREV', axis = 1), col = 'SK_ID_CURR', name = 'PREVIOUA_APP')
prev_app_agg_cat = agg_fn_categorical(data = previous_app, col = 'SK_ID_CURR', name = 'PREVIOUS_APP')

<ipython-input-26-bcb3b9189408>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[col] = group_ids


In [68]:
previous_app_final = prev_app_agg_cat.merge(prev_app_agg_conti, on = 'SK_ID_CURR', how = 'outer')

In [86]:
pos_cash_bal_final = double_agg_fn(pos_cash_bal, ['SK_ID_CURR', 'SK_ID_PREV'], ['CLIENT', 'POS_CASH'], pos_cash_bal)

<ipython-input-26-bcb3b9189408>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[col] = group_ids


In [81]:
credit_card_bal_final = double_agg_fn(credit_card_bal, ['SK_ID_CURR', 'SK_ID_PREV'], ['CLIENT', 'CREDIT_CARD'], credit_card_bal)

<ipython-input-26-bcb3b9189408>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[col] = group_ids


,SK_ID_CURR,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_NORM_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_NORM_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_NORM_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_NORM_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_ACTIVE_COUNT_NORM_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_NORM_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_NORM_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_NORM_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_NORM_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_APPROVED_COUNT_NORM_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_NORM_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_NORM_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_NORM_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_NORM_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_COMPLETED_COUNT_NORM_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_NORM_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_NORM_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_NORM_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_NORM_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_DEMAND_COUNT_NORM_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_NORM_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_NORM_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_NORM_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_NORM_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_REFUSED_COUNT_NORM_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_NORM_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_NORM_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_NORM_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_NORM_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SENT PROPOSAL_COUNT_NORM_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SIGNED_COUNT_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SIGNED_COUNT_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SIGNED_COUNT_MAX,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SIGNED_COUNT_MIN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SIGNED_COUNT_SUM,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SIGNED_COUNT_NORM_COUNT,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_SIGNED_COUNT_NORM_MEAN,CLIENT_POS_CASH_NAME_CONTRACT_STATUS_S

In [91]:
installment_payments_numeric = agg_fn_numeric(installment_payments, 'SK_ID_PREV', 'INSTALLMENT')

In [93]:
installment_payments_merged = installment_payments_numeric.merge(installment_payments[['SK_ID_PREV', 'SK_ID_CURR']], how = 'left', on = 'SK_ID_PREV')

In [95]:
installment_payments_final = agg_fn_numeric(installment_payments_merged.drop('SK_ID_PREV', axis = 1), 'SK_ID_CURR', 'CLIENT')

In [98]:
train = train.merge(bureau_final, on = 'SK_ID_CURR', how = 'left')
test = test.merge(bureau_final, on = 'SK_ID_CURR', how = 'left')
train = train.merge(bureau_bal_final, on = 'SK_ID_CURR', how = 'left')
test = test.merge(bureau_bal_final, on = 'SK_ID_CURR', how = 'left')
train = train.merge(previous_app_final, on = 'SK_ID_CURR', how = 'left')
test = test.merge(previous_app_final, on = 'SK_ID_CURR', how = 'left')
train = train.merge(pos_cash_bal_final, on = 'SK_ID_CURR', how = 'left')
test = test.merge(pos_cash_bal_final, on = 'SK_ID_CURR', how = 'left')
train = train.merge(credit_card_bal_final, on = 'SK_ID_CURR', how = 'left')
test = test.merge(credit_card_bal_final, on = 'SK_ID_CURR', how = 'left')
train = train.merge(installment_payments_final, on = 'SK_ID_CURR', how = 'left')
test = test.merge(installment_payments_final, on = 'SK_ID_CURR', how = 'left')

In [99]:
train.shape

(307511, 1644)

In [100]:
test.shape

(48744, 1643)

In [102]:
train.to_csv('train_merged.csv', index = False)
test.to_csv('test_merged.csv', index = False)